# Introduction

All student names in group: [Liana Torpey, Blake Lewinski]

I understand that my submission needs to be my own group's work: [LGT, BCL]

I understand that ChatGPT / Copilot / other AI tools are not allowed: [LGT, BCL]

Points: 10

Complete this notebook and submit it (save/print as pdf). Make sure all output is correct in the pdf before submitting (it sometimes gets cut off). The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [ ]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 12.0 MB/s eta 0:00:00


In [ ]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as the variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constaints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(1 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(1 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(1 point)**

Solution: boolean formulas for the constraints that the number...

5 can occur at most once in the first row<br>
$\neg X_{1,1}^5 \lor \neg X_{1,2}^5 \lor \neg X_{1,3}^5 \lor \neg X_{1,4}^5 \lor \neg X_{1,5}^5 \lor \neg X_{1,6}^5 \lor \neg X_{1,7}^5 \lor \neg X_{1,8}^5 \lor \neg X_{1,9}^5
$

6 can occur at most once in the first column<br>
$\neg X_{1,1}^6 \lor \neg X_{2,1}^6 \lor \neg X_{3,1}^6 \lor \neg X_{4,1}^6 \lor \neg X_{5,1}^6 \lor \neg X_{6,1}^6 \lor \neg X_{7,1}^6 \lor \neg X_{8,1}^6 \lor \neg X_{9,1}^6
$

9 can occur at most once in the top left 3x3 subgrid<br>
$\neg X_{1,1}^9 \lor \neg X_{1,2}^9 \lor \neg X_{1,3}^9 \lor \neg X_{2,1}^9 \lor \neg X_{2,2}^9 \lor \neg X_{2,3}^9 \lor \neg X_{3,1}^9 \lor \neg X_{3,2}^9 \lor \neg X_{3,3}^9
$

## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(1 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [ ]:
from z3 import *

# Define the Sudoku instance
instance = [
    [None, 1, None, 4, None, 2, None, 5, None],
    [5, None, None, None, None, None, None, None, 6],
    [None, None, None, 3, None, 1, None, None, None],
    [7, None, 5, None, None, None, 4, None, 8],
    [None, None, None, None, None, None, None, None, None],
    [2, None, 8, None, None, None, 5, None, 9],
    [None, None, None, 9, None, 6, None, None, None],
    [6, None, None, None, None, None, None, None, 2],
    [None, 7, None, 1, None, 3, None, 4, None]
]

# Create a Z3 solver
solver = Solver()

# Create boolean variables for each cell in the Sudoku grid
X = [[[Bool(f'X_{i}_{j}_{k}') for k in range(9)] for j in range(9)] for i in
        range(9)]

# Add constraints for the known values in the Sudoku instance
for i in range(9):
    for j in range(9):
        if instance[i][j] is not None:
            for k in range(9):
                if k != instance[i][j] - 1:
                    solver.add(Not(X[i][j][k]))

# Add constraints to ensure each cell contains exactly one value
for i in range(9):
    for j in range(9):
        solver.add(Or(X[i][j]))

# Add constraints to ensure each row contains distinct values
for i in range(9):
    for k in range(9):
        solver.add(Or([X[i][j][k] for j in range(9)]))
        for j in range(9):
            for l in range(j + 1, 9):
                solver.add(Or(Not(X[i][j][k]), Not(X[i][l][k])))

# Add constraints to ensure each column contains distinct values
for j in range(9):
    for k in range(9):
        solver.add(Or([X[i][j][k] for i in range(9)]))
        for i in range(9):
            for l in range(i + 1, 9):
                solver.add(Or(Not(X[i][j][k]), Not(X[l][j][k])))

# Add constraints to ensure each 3x3 subgrid contains distinct values
for r in range(3):
    for c in range(3):
        for k in range(9):
            solver.add(Or([X[3 * r + i][3 * c + j][k] for i in range(3) for j
                           in range(3)]))
            for i in range(3):
                for j in range(3):
                    for l in range(j + 1, 3):
                        solver.add(Or(Not(X[3 * r + i][3 * c + j][k]),
                                      Not(X[3 * r + i][3 * c + l][k])))
                    for l in range(i + 1, 3):
                        solver.add(Or(Not(X[3 * r + i][3 * c + j][k]),
                                      Not(X[3 * r + l][3 * c + j][k])))

# Check if the Sudoku instance is solvable
if solver.check() == sat:
    # Get the solution
    model = solver.model()
    # Print the solution
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)  # Separate 3x3 boxes horizontally
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")  # Separate 3x3 boxes vertically
            solved = False
            for k in range(9):
                if is_true(model.evaluate(X[i][j][k])):
                    if instance[i][j] is not None:
                        print(f'\033[94m{instance[i][j]}\033[0m', end=' ')
                            # Initial numbers in blue
                    else:
                        print(f'\033[30m{k+1}\033[0m', end=' ')
                            # Solved new numbers in black
                    solved = True
                    break
            if not solved:
                print("X", end=' ')
        print()
else:
    print("The Sudoku instance is unsolvable.")


3 1 9 | 4 6 2 | 8 5 7 
5 2 4 | 7 9 8 | 1 3 6 
8 6 7 | 3 5 1 | 9 2 4 
---------------------
7 3 5 | 2 1 9 | 4 6 8 
1 9 6 | 8 4 5 | 2 7 3 
2 4 8 | 6 3 7 | 5 1 9 
---------------------
4 5 3 | 9 2 6 | 7 8 1 
6 8 1 | 5 7 4 | 3 9 2 
9 7 2 | 1 8 3 | 6 4 5 


### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(1 point)**

To check if the solution is unique, modify the code to continue searching for alternative solutions after finding one. If find another solution, then the puzzle is not unique. If no additional solutions are found, then the puzzle is indeed unique.

In [ ]:
from z3 import *

# Define the Sudoku instance
instance = [
    [None, 1, None, 4, None, 2, None, 5, None],
    [5, None, None, None, None, None, None, None, 6],
    [None, None, None, 3, None, 1, None, None, None],
    [7, None, 5, None, None, None, 4, None, 8],
    [None, None, None, None, None, None, None, None, None],
    [2, None, 8, None, None, None, 5, None, 9],
    [None, None, None, 9, None, 6, None, None, None],
    [6, None, None, None, None, None, None, None, 2],
    [None, 7, None, 1, None, 3, None, 4, None]
]

# Create a Z3 solver
solver = Solver()

# Create boolean variables for each cell in the Sudoku grid
X = [[[Bool(f'X_{i}_{j}_{k}') for k in range(9)] for j in range(9)] for i
        in range(9)]

# Add constraints for the known values in the Sudoku instance
for i in range(9):
    for j in range(9):
        if instance[i][j] is not None:
            for k in range(9):
                if k != instance[i][j] - 1:
                    solver.add(Not(X[i][j][k]))

# Add constraints to ensure each cell contains exactly one value
for i in range(9):
    for j in range(9):
        solver.add(Or(X[i][j]))

# Add constraints to ensure each row contains distinct values
for i in range(9):
    for k in range(9):
        solver.add(Or([X[i][j][k] for j in range(9)]))
        for j in range(9):
            for l in range(j + 1, 9):
                solver.add(Or(Not(X[i][j][k]), Not(X[i][l][k])))

# Add constraints to ensure each column contains distinct values
for j in range(9):
    for k in range(9):
        solver.add(Or([X[i][j][k] for i in range(9)]))
        for i in range(9):
            for l in range(i + 1, 9):
                solver.add(Or(Not(X[i][j][k]), Not(X[l][j][k])))

# Add constraints to ensure each 3x3 subgrid contains distinct values
for r in range(3):
    for c in range(3):
        for k in range(9):
            solver.add(Or([X[3 * r + i][3 * c + j][k] for i in range(3) for j
                           in range(3)]))
            for i in range(3):
                for j in range(3):
                    for l in range(j + 1, 3):
                        solver.add(Or(Not(X[3 * r + i][3 * c + j][k]),
                                      Not(X[3 * r + i][3 * c + l][k])))
                    for l in range(i + 1, 3):
                        solver.add(Or(Not(X[3 * r + i][3 * c + j][k]),
                                      Not(X[3 * r + l][3 * c + j][k])))

# Check if the Sudoku instance is solvable
if solver.check() == sat:
    # Get the solution
    model = solver.model()
    # Print the solution
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)  # Separate 3x3 boxes horizontally
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")  # Separate 3x3 boxes vertically
            for k in range(9):
                if is_true(model.evaluate(X[i][j][k])):
                    if instance[i][j] is not None:
                        print(f"\033[34m{instance[i][j]}\033[0m", end=' ')
                            # Blue color for initial numbers
                    else:
                        print(f"\033[30m{k + 1}\033[0m", end=' ')
                            # Black color for solved numbers
        print()

    # Check for additional solutions
    count = 0
    while solver.check() == sat:
        if count == 1:
            print("More than one solution exists, therefore the puzzle is not \
                unique.")
        count += 1
        # Get the next solution
        model = solver.model()
        # Exclude the current solution
        block = []
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    if is_true(model.evaluate(X[i][j][k])):
                        block.append(X[i][j][k] != True)
        solver.add(Or(block))
    print("Number of solutions found:", count)
    if count == 1:
        print("The puzzle is unique, only one solution has been found.")
else:
    print("The Sudoku instance is unsolvable.")

3 1 9 | 4 6 2 | 8 5 7 
5 2 4 | 7 9 8 | 1 3 6 
8 6 7 | 3 5 1 | 9 2 4 
---------------------
7 3 5 | 2 1 9 | 4 6 8 
1 9 6 | 8 4 5 | 2 7 3 
2 4 8 | 6 3 7 | 5 1 9 
---------------------
4 5 3 | 9 2 6 | 7 8 1 
6 8 1 | 5 7 4 | 3 9 2 
9 7 2 | 1 8 3 | 6 4 5 
Number of solutions found: 1
The puzzle is unique, only one solution has been found.


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be less than 5000. **(1 point)**

In [ ]:
from z3 import *

# Define the Sudoku instance - REMOVED THE 1 IN THE TOP LEFT 3X3 ROW
instance = [
    [None, None, None, 4, None, 2, None, 5, None],
    [5, None, None, None, None, None, None, None, 6],
    [None, None, None, 3, None, 1, None, None, None],
    [7, None, 5, None, None, None, 4, None, 8],
    [None, None, None, None, None, None, None, None, None],
    [2, None, 8, None, None, None, 5, None, 9],
    [None, None, None, 9, None, 6, None, None, None],
    [6, None, None, None, None, None, None, None, 2],
    [None, 7, None, 1, None, 3, None, 4, None]
]

# Create a Z3 solver
solver = Solver()

# Create boolean variables for each cell in the Sudoku grid
X = [[[Bool(f'X_{i}_{j}_{k}') for k in range(9)] for j in range(9)] for i in
        range(9)]

# Add constraints for the known values in the Sudoku instance
for i in range(9):
    for j in range(9):
        if instance[i][j] is not None:
            for k in range(9):
                if k != instance[i][j] - 1:
                    solver.add(Not(X[i][j][k]))

# Add constraints to ensure each cell contains exactly one value
for i in range(9):
    for j in range(9):
        solver.add(Or(X[i][j]))

# Add constraints to ensure each row contains distinct values
for i in range(9):

    for k in range(9):
        solver.add(Or([X[i][j][k] for j in range(9)]))
        for j in range(9):
            for l in range(j + 1, 9):
                solver.add(Or(Not(X[i][j][k]), Not(X[i][l][k])))

# Add constraints to ensure each column contains distinct values
for j in range(9):
    for k in range(9):
        solver.add(Or([X[i][j][k] for i in range(9)]))
        for i in range(9):
            for l in range(i + 1, 9):
                solver.add(Or(Not(X[i][j][k]), Not(X[l][j][k])))

# Add constraints to ensure each 3x3 subgrid contains distinct values
for r in range(3):
    for c in range(3):
        for k in range(9):
            solver.add(Or([X[3 * r + i][3 * c + j][k] for i in range(3) for j
                           in range(3)]))
            for i in range(3):
                for j in range(3):
                    for l in range(j + 1, 3):
                        solver.add(Or(Not(X[3 * r + i][3 * c + j][k]),
                                      Not(X[3 * r + i][3 * c + l][k])))
                    for l in range(i + 1, 3):
                        solver.add(Or(Not(X[3 * r + i][3 * c + j][k]),
                                      Not(X[3 * r + l][3 * c + j][k])))

# Check if the Sudoku instance is solvable
if solver.check() == sat:
    # Get the solution
    model = solver.model()
    # Print the solution
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)  # Separate 3x3 boxes horizontally
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")  # Separate 3x3 boxes vertically
            for k in range(9):
                if is_true(model.evaluate(X[i][j][k])):
                    if instance[i][j] is not None:
                        print(f"\033[94m{instance[i][j]}\033[0m", end=' ')
                            # Print initial numbers in blue
                    else:
                        print(f"\033[30m{k + 1}\033[0m", end=' ')
                            # Print solved numbers in black
        print()

    # Check for additional solutions
    count = 0
    while solver.check() == sat:
        if count == 1 :
            print("More than one solution exists, therefore the puzzle is not"
            + " unique.")
        count += 1
        # Get the next solution
        model = solver.model()
        # Exclude the current solution
        block = []
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    if is_true(model.evaluate(X[i][j][k])):
                        block.append(X[i][j][k] != True)
        solver.add(Or(block))
    print("Number of solutions found:", count)
    if count == 1 :
        print("The puzzle is unique, only one solution has been found.")
else:
    print("The Sudoku instance is unsolvable.")


1 8 3 | 4 6 2 | 9 5 7 
5 2 4 | 8 9 7 | 1 3 6 
9 6 7 | 3 5 1 | 8 2 4 
---------------------
7 1 5 | 2 3 9 | 4 6 8 
4 9 6 | 5 1 8 | 2 7 3 
2 3 8 | 6 7 4 | 5 1 9 
---------------------
3 5 2 | 9 4 6 | 7 8 1 
6 4 1 | 7 8 5 | 3 9 2 
8 7 9 | 1 2 3 | 6 4 5 
More than one solution exists, therefore the puzzle is not unique.
Number of solutions found: 200


# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(2 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [ ]:
from z3 import *

# Define the Miracle Sudoku instance
instance = [
    [None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None, None],
    [None, None, 1, None, None, None, None, None, None],
    [None, None, None, None, None, None, 2, None, None],
    [None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None, None, None]
]


# Create a Z3 solver
solver = Solver()

# Create boolean variables for each cell in the Sudoku grid
X = [[[Bool(f'X_{i}_{j}_{k}') for k in range(9)] for j in range(9)] for i
        in range(9)]

# Add constraints for the known values in the Sudoku instance
for i in range(9):
    for j in range(9):
        if instance[i][j] is not None:
            for k in range(9):
                if k != instance[i][j] - 1:
                    solver.add(Not(X[i][j][k]))

# Add constraints to ensure each cell contains exactly one value
for i in range(9):
    for j in range(9):
        solver.add(Or(X[i][j]))

# Add constraints to ensure each row contains distinct values
for i in range(9):
    for k in range(9):
        solver.add(Or([X[i][j][k] for j in range(9)]))
        for j in range(9):
            for l in range(j + 1, 9):
                solver.add(Or(Not(X[i][j][k]), Not(X[i][l][k])))

# Add constraints to ensure each column contains distinct values
for j in range(9):
    for k in range(9):
        solver.add(Or([X[i][j][k] for i in range(9)]))
        for i in range(9):
            for l in range(i + 1, 9):
                solver.add(Or(Not(X[i][j][k]), Not(X[l][j][k])))

# Add constraints to ensure each 3x3 subgrid contains distinct values
for r in range(3):
    for c in range(3):
        for k in range(9):
            solver.add(Or([X[3 * r + i][3 * c + j][k] for i in range(3) for j
                           in range(3)]))
            for i in range(3):


                for j in range(3):
                    for l in range(j + 1, 3):
                        solver.add(Or(Not(X[3 * r + i][3 * c + j][k]),
                                      Not(X[3 * r + i][3 * c + l][k])))
                    for l in range(i + 1, 3):
                        solver.add(Or(Not(X[3 * r + i][3 * c + j][k]),
                                      Not(X[3 * r + l][3 * c + j][k])))

# Add constraints for knight's move
for i in range(9):
    for j in range(9):
        for dx, dy in [(-2, -1), (-2, 1), (-1, -2), (-1, 2), (1, -2), (1, 2),
        (2, -1), (2, 1)]:
            ni, nj = i + dx, j + dy
            if 0 <= ni < 9 and 0 <= nj < 9:
                for k in range(9):
                    solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k])))

# Add constraints for king's move
for i in range(9):
    for j in range(9):
        for dx, dy in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1),
         (1, 0), (1, 1)]:
            ni, nj = i + dx, j + dy
            if 0 <= ni < 9 and 0 <= nj < 9:
                for k in range(9):
                    solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k])))

# Add constraints for consecutive digits
for i in range(9):
    for j in range(9):
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            ni, nj = i + dx, j + dy
            if 0 <= ni < 9 and 0 <= nj < 9:
                for k in range(9):
                    solver.add(Or(Not(X[i][j][k]),
                                  Not(X[ni][nj][(k + 1) % 9])))

# Check if the Sudoku instance is solvable
if solver.check() == sat:
    # Get the solution
    model = solver.model()
    # Print the solution
    for i in range(9):

        if i % 3 == 0 and i != 0:
            print("-" * 21)  # Separate 3x3 boxes horizontally
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")  # Separate 3x3 boxes vertically
            for k in range(9):
                if is_true(model.evaluate(X[i][j][k])):
                    if instance[i][j] is not None:
                        if instance[i][j] == k + 1:
                            print("\033[94m" + str(k + 1), end=' ')
                        else:
                            print("\033[0m" + str(k + 1), end=' ')
                    else:
                        print("\033[0m" + str(k + 1), end=' ')
        print()

    # Check for additional solutions
    count = 0
    while solver.check() == sat:
        count += 1
        # Get the next solution
        model = solver.model()
        # Exclude the current solution
        block = []
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    if is_true(model.evaluate(X[i][j][k])):
                        block.append(X[i][j][k] != True)
        solver.add(Or(block))
    print("Number of solutions found:", count)
    if count == 1:
        print("The puzzle is unique, only one solution has been found.")
    else:
        print("More than one solution exists, therefore the puzzle is not \
            unique.")
else:
    print("The Sudoku instance is unsolvable.")


4 8 3 | 7 2 6 | 1 5 9 
7 2 6 | 1 5 9 | 4 8 3 
1 5 9 | 4 8 3 | 7 2 6 
---------------------
8 3 7 | 2 6 1 | 5 9 4 
2 6 1 | 5 9 4 | 8 3 7 
5 9 4 | 8 3 7 | 2 6 1 
---------------------
3 7 2 | 6 1 5 | 9 4 8 
6 1 5 | 9 4 8 | 3 7 2 
9 4 8 | 3 7 2 | 6 1 5 
Number of solutions found: 1
The puzzle is unique, only one solution has been found.




---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(1 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$

| p | q | r | s | t | p → q | (p → q) → r | s → ¬p | ¬s | t → q |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
|T      |T      |T      |T      |T      |T      |T      |F      |F      |T
|T      |T      |T      |T      |F      |T      |T      |F      |F      |T
|**T**      |**T**      |**T**      |**F**      |**T**      |**T**      |**T**      |**T**      |**T**      |**T**
|T      |T      |T      |F      |F      |T      |T      |T      |T      |T
|T      |T      |F      |T      |T      |T      |F      |F      |F      |T
|T      |T      |F      |T      |F      |T      |F      |F      |F      |T
|T      |T      |F      |F      |T      |T      |F      |T      |T      |T
|T      |T      |F      |F      |F      |T      |F      |T      |T      |T
|T      |F      |T      |T      |T      |F      |T      |F      |F      |F
|T      |F      |T      |T      |F      |F      |T      |F      |F      |T
|T      |F      |T      |F      |T      |F      |T      |T      |T      |F
|T      |F      |T      |F      |F      |F      |T      |T      |T      |T
|T      |F      |F      |T      |T      |F      |T      |F      |F      |F
|T      |F      |F      |T      |F      |F      |T      |F      |F      |T
|T      |F      |F      |F      |T      |F      |T      |T      |T      |F
|T      |F      |F      |F      |F      |F      |T      |T      |T      |T
|F      |T      |T      |T      |T      |T      |T      |T      |F      |T
|F      |T      |T      |T      |F      |T      |T      |T      |F      |T
|**F**      |**T**      |**T**      |**F**      |**T**      |**T**      |**T**      |**T**      |**T**      |**T**
|F      |T      |T      |F      |F      |T      |T      |T      |T      |T
|F      |T      |F      |T      |T      |T      |F      |T      |F      |T
|F      |T      |F      |T      |F      |T      |F      |T      |F      |T
|F      |T      |F      |F      |T      |T      |F      |T      |T      |T
|F      |T      |F      |F      |F      |T      |F      |T      |T      |T
|F      |F      |T      |T      |T      |T      |T      |T      |F      |F
|F      |F      |T      |T      |F      |T      |T      |T      |F      |T
|F      |F      |T      |F      |T      |T      |T      |T      |T      |F
|F      |F      |T      |F      |F      |T      |T      |T      |T      |T
|F      |F      |F      |T      |T      |T      |F      |T      |F      |F
|F      |F      |F      |T      |F      |T      |F      |T      |F      |T
|F      |F      |F      |F      |T      |T      |F      |T      |T      |F
|F      |F      |F      |F      |F      |T      |F      |T      |T      |T

We found two models that prove the sentence holds, in rows 3 and 19. In these rows we can see that $r$ is true when $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s$ and $t \rightarrow q$ are true.

 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$

| p | q | q → p | p → (q → p) | p → (p → (q → p)) | q → (p → (p → (q → p)))
|-------|-------|-------|-------|-------|-------
|**T**     |**T**      |**T**      |**T**      |**T**      |**T**
|**T**     |**F**      |**T**      |**T**      |**T**      |**T**
|**F**     |**T**      |**F**      |**T**      |**T**      |**T**
|**F**     |**F**      |**T**      |**T**      |**T**      |**T**

Empty set entails tautologies. Since every model proved to be true, the entailment holds; thus we bolded every row.


 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$

| p | q | r | s | t | p → q | (p → q) → r | s → ¬p | ¬s | t → q |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
|T      |T      |T      |T      |T      |T      |T      |F      |F      |T
|T      |T      |T      |T      |F      |T      |T      |F      |F      |T
|**T**      |**T**      |**T**      |**F**      |**T**      |**T**      |**T**      |**T**      |**T**      |**T**
|T      |T      |T      |F      |F      |T      |T      |T      |T      |T
|T      |T      |F      |T      |T      |T      |F      |F      |F      |T
|T      |T      |F      |T      |F      |T      |F      |F      |F      |T
|T      |T      |F      |F      |T      |T      |F      |T      |T      |T
|T      |T      |F      |F      |F      |T      |F      |T      |T      |T
|T      |F      |T      |T      |T      |F      |T      |F      |F      |F
|T      |F      |T      |T      |F      |F      |T      |F      |F      |T
|T      |F      |T      |F      |T      |F      |T      |T      |T      |F
|T      |F      |T      |F      |F      |F      |T      |T      |T      |T
|T      |F      |F      |T      |T      |F      |T      |F      |F      |F
|T      |F      |F      |T      |F      |F      |T      |F      |F      |T
|T      |F      |F      |F      |T      |F      |T      |T      |T      |F
|T      |F      |F      |F      |F      |F      |T      |T      |T      |T
|F      |T      |T      |T      |T      |T      |T      |T      |F      |T
|F      |T      |T      |T      |F      |T      |T      |T      |F      |T
|**F**      |**T**      |**T**      |**F**      |**T**      |**T**      |**T**      |**T**      |**T**      |**T**
|F      |T      |T      |F      |F      |T      |T      |T      |T      |T
|F      |T      |F      |T      |T      |T      |F      |T      |F      |T
|F      |T      |F      |T      |F      |T      |F      |T      |F      |T
|F      |T      |F      |F      |T      |T      |F      |T      |T      |T
|F      |T      |F      |F      |F      |T      |F      |T      |T      |T
|F      |F      |T      |T      |T      |T      |T      |T      |F      |F
|F      |F      |T      |T      |F      |T      |T      |T      |F      |T
|F      |F      |T      |F      |T      |T      |T      |T      |T      |F
|F      |F      |T      |F      |F      |T      |T      |T      |T      |T
|F      |F      |F      |T      |T      |T      |F      |T      |F      |F
|F      |F      |F      |T      |F      |T      |F      |T      |F      |T
|F      |F      |F      |F      |T      |T      |F      |T      |T      |F
|F      |F      |F      |F      |F      |T      |F      |T      |T      |T

We found two models that prove the sentence holds, in rows 3 and 19. In these rows we can see that $r$ is true when $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s$ and $t \rightarrow q$ are true.

(add markdown tables here)



---


 ## Logical Entailment using z3py

 Use Z3py to prove the above three entailments are true using satisfiability. See section 7.5 in the book for details. **(1 point)**

In [ ]:
from z3 import *

# Create boolean variables
p, q, r, s, t = Bools('p q r s t')

# First entailment: (p → q) → r, s → ¬p, t, ¬s, t → q |= r
entailment1 = Implies(And(Implies(Implies(p, q), r), s, Not(p), t, Not(s),
                          Implies(t, q)), r)

# Second entailment: ∅ |= q → (p → (p → (q → p)))
entailment2 = Implies(True, Implies(q, Implies(p, Implies(p, Implies(q, p)))))

# Third entailment: (p → q) → r, s → ¬p, t, ¬s, t → q |= r
entailment3 = Implies(And(Implies(Implies(p, q), r), s, Not(p), t, Not(s),
                          Implies(t, q)), r)

# Check if the negation of the entailments is unsatisfiable
solver = Solver()
solver.add(Not(entailment1))
solver.add(Not(entailment2))
solver.add(Not(entailment3))

# Refutation involves attempting to show that the negation of the entailment
# is unsatisfiable. If the negation is unsatisfiable, it implies that the
# original entailment is valid.

if solver.check() == unsat:
    print("All three entailments are true.")
else: # Proves with counter example
    print("At least one of the entailments is not true.")


All three entailments are true.
